# NLP Final Project 2024

## Introduction
This research investigates if advanced language models can act as rational agents in simulated second-price auctions (the winner pays the second-highest bid) by strategically bidding their true value based on limited information, as economic theory predicts for rational agents.


In this work, we will use the following open-source LLMs:

1. Meta-Llama-3.1-8B - https://huggingface.co/meta-llama/Meta-Llama-3.1-8B
2. Mistral -https://huggingface.co/mistralai/Mistral-7B-v0.1
3. T5 - https://huggingface.co/google/flan-t5-large

In [ ]:
!pip install --upgrade transformers
!pip install accelerate transformers
!pip install -U flash-attn --no-build-isolation
!pip install transformers bitsandbytes
!pip install accelerate

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline, AutoModelForCausalLM, AutoTokenizer,T5Tokenizer, T5ForConditionalGeneration
import torch

from huggingface_hub import login
import os
import json

from google.colab import files
import torch

torch.cuda.empty_cache()

In [ ]:
# logging into huggingface
login()

### Promts

In [ ]:
# Convert test prompt to a single string
def format_prompt(prompt_list):
    formatted_prompt = ""
    for prompt in prompt_list:
        formatted_prompt += f"{prompt['role']}: {prompt['content']}\n"
    return formatted_prompt

In [ ]:
# Zero shot
small_item_zero = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": "Your value for the item is 1 USD. This is all the information you have (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]


medium_item_zero = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participant is only exposed to his own bid.'},
    {"role": "user", "content": "you value from the item is 1000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info.(2) briefly explain your reasoning."}
]

big_item_zero = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participant is only exposed to his own bid.'},
    {"role": "user", "content": "you value from the item is 10,000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info.(2) briefly explain your reasoning."}
]


giant_item_zero = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participant is only exposed to his own bid.'},
    {"role": "user", "content": "you value from the item is 10,000,000,000,000,000,000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info.(2) briefly explain your reasoning."}
]

In [ ]:
# Zero shot
formatted_small_zero = format_prompt(small_item_zero)
formatted_medium_zero = format_prompt(medium_item_zero)
formatted_big_zero = format_prompt(big_item_zero)
formatted_giant_zero = format_prompt(giant_item_zero)

In [ ]:
# One shot
small_item_one = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": "Your value for the item is 1 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."},
    {"role": "assistant", "content": "I will bid 1 USD. My reasoning is that in a second-price auction, bidding your true valuation maximizes your expected utility, as you pay the second-highest bid if you win."},
    {"role": "user", "content": "Your value for the item is 2 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]

medium_item_one = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": "You value the item at 1000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."},
    {"role": "assistant", "content": "I will bid 1000 USD. My reasoning is that in a second-price auction, bidding your true valuation ensures that if you win, you pay a price lower than or equal to your valuation, maximizing your utility."},
    {"role": "user", "content": "You value the item at 1500 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]

big_item_one = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": "You value the item at 10,000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."},
    {"role": "assistant", "content": "I will bid 10,000 USD. My reasoning is that in a second-price auction, bidding your true valuation ensures that you maximize your utility by either winning at a favorable price or not winning if the price is too high."},
    {"role": "user", "content": "You value the item at 15,000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]

giant_item_one = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": "You value the item at 10,000,000,000,000,000,000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."},
    {"role": "assistant", "content": "I will bid 10,000,000,000,000,000,000 USD. My reasoning is that bidding your true valuation in a second-price auction ensures that you either win at a price you are willing to pay or do not win at all, which maximizes your expected utility."},
    {"role": "user", "content": "You value the item at 15,000,000,000,000,000,000 USD. This is all the information you have. (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]

In [ ]:
formatted_small_one = format_prompt(small_item_one)
formatted_medium_one = format_prompt(medium_item_one)
formatted_big_one = format_prompt(big_item_one)
formatted_giant_one = format_prompt(giant_item_one)

In [ ]:
# items

dental_floss = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": "The item: Plackers Micro Mint Dental Floss Picks with Travel Case, 12 Count. Your value for the item is 1 USD. This is all the information you have (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]


speaker = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": " The item: Google Home White Slate One Size Smart Speaker. Your value for the item is 116 USD. This is all the information you have (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]


rolex = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": " The item: Rolex Oyster Perpetual 41mm Automatic Blue Dial Men's Watch. Your value for the item is 9850 USD. This is all the information you have (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]

spaceship = [
    {"role": "system", "content": 'You are a bidder, participating in a second-price auction, where each bidder maximizes their expected utility by bidding their valuation of the item for sale. Each of the X participants is only exposed to their own bid.'},
    {"role": "user", "content": " The item: a large and impressive spaceship. Your value for the item is 10,000,000,000,000,000,000 USD. This is all the information you have (1) How much will you bid? Just return your bid and no other info. (2) Briefly explain your reasoning."}
]

In [ ]:
formatted_dental_floss = format_prompt(dental_floss)
formatted_speaker = format_prompt(speaker)
formatted_rolex = format_prompt(rolex)
formatted_spaceship = format_prompt(spaceship)

In [ ]:
N = 10 # Number of responses we are looking to get

In [ ]:
# Ensure GPU is selected in Colab runtime
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


### *Mistral*

In [ ]:
# Path to offload directory
offload_dir = "/content/offload_dir"

# Create offload directory if it doesn't exist
os.makedirs(offload_dir, exist_ok=True)

# Load the model
Mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=offload_dir
)

# Load the tokenizer
Mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [ ]:
def mistral_generate(prompt):
    Mistral_model_inputs = Mistral_tokenizer(prompt, return_tensors="pt")

    # Ensure the model inputs are on the same device as the model
    Mistral_model_inputs = {k: v.to(device) for k, v in Mistral_model_inputs.items()}

    # Generate text
    Mistral_generated_ids = Mistral_model.generate(**Mistral_model_inputs, max_new_tokens=150, do_sample=True, temperature=1,top_k=50,top_p=0.9)

    # Decode & return
    output_Mistral = Mistral_tokenizer.batch_decode(Mistral_generated_ids, skip_special_tokens=True)[0]
    return output_Mistral

### T5

In [ ]:
# Load the tokenizer and model
T5_model = "google/flan-t5-large"
T5_tokenizer = T5Tokenizer.from_pretrained(T5_model)
model = T5ForConditionalGeneration.from_pretrained(T5_model)
model.to(device)

In [ ]:
def T5_generate(prompt):

    # Tokenize the prompt
    T5_inputs = T5_tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the output with group beam search and diversity penalty
    outputs = model.generate(
        T5_inputs["input_ids"],
        max_length=150,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7,
    )

    # Decode the generated text
    generated_text = T5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


### Llama 3.1

In [ ]:
# Define the model ID and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load the model and tokenizer with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,  # Enable 4-bit quantization
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create a pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
def llama_generate(prompt):
    try:
        # Generate text
        outputs = pipe(
            prompt,
            max_new_tokens=150,
            eos_token_id=pipe.tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

        # Return the generated text
        if outputs:
            return outputs[0]['generated_text'][len(prompt):]
        else:
            return "No output generated."
    except Exception as e:
        return f"Error during text generation: {e}"

### Responses

In [ ]:
def model_response(prompt, N, item_size, model_name):
    res = []
    function_name = None

    if model_name == 'Mistral':
        function_name = mistral_generate
    elif model_name == 'Llama':
        function_name = llama_generate
    elif model_name == 'T5':
        function_name = T5_generate
    else:
        print('Model name is not valid')
        return

    for i in range(N):
        response = function_name(prompt)
        res.append(response)
        print(f'Response {i+1}:')
        print(response)
        print('------------------')


    # Save results to a JSON file
    output_file = f"{model_name}_responses_{item_size}.json"
    with open(output_file, "w") as f:
        json.dump(res, f, indent=4)

    files.download(output_file)
    print(f"Results saved to {output_file}")

    print('------------------------------')

### small item

In [ ]:
model_response(formatted_small_zero, N, 'small', 'Mistral')

In [ ]:
model_response(formatted_small_one, N, 'small_one', 'Mistral')

In [ ]:
model_response(formatted_small_zero, N, 'small', 'Llama')

In [ ]:
model_response(formatted_small_one, N, 'small_one', 'Llama')

In [ ]:
model_response(formatted_small_zero, N, 'small', 'T5')

In [ ]:
model_response(formatted_small_one, N, 'small_one', 'T5')

### medium item

In [ ]:
model_response(formatted_medium_zero, N, 'medium','Mistral')

In [ ]:
model_response(formatted_medium_one, N, 'medium_one','Mistral')

In [ ]:
model_response(formatted_medium_zero, N, 'medium', 'Llama')

In [ ]:
model_response(formatted_medium_one, N, 'medium_one', 'Llama')

In [ ]:
model_response(formatted_medium_zero, N, 'medium', 'T5')

In [ ]:
model_response(formatted_medium_one, N, 'medium_one', 'T5')

### big item

In [ ]:
model_response(formatted_big_zero, N, 'big', 'Mistral')

In [ ]:
model_response(formatted_big_one, N, 'big_one', 'Mistral')

In [ ]:
model_response(formatted_big_zero, N, 'big', 'Llama')

In [ ]:
model_response(formatted_big_one, N, 'big_one', 'Llama')

In [ ]:
model_response(formatted_big_zero, N, 'big', 'T5')

In [ ]:
model_response(formatted_big_one, N, 'big_one', 'T5')

### giant item

In [ ]:
model_response(formatted_giant_zero, N, 'giant', 'Mistral')

In [ ]:
model_response(formatted_giant_one, N, 'giant_one', 'Mistral')

In [ ]:
model_response(formatted_giant_zero, N, 'giant', 'Llama')

In [ ]:
model_response(formatted_giant_one, N, 'giant_one', 'Llama')

In [ ]:
model_response(formatted_giant_zero, N, 'giant', 'T5')

In [ ]:
model_response(formatted_giant_one, N, 'giant_one', 'T5')

### items

In [ ]:
model_response(formatted_dental_floss, N, 'dental_floss', 'Llama')

In [ ]:
model_response(formatted_speaker, N, 'speaker', 'Llama')

In [ ]:
model_response(formatted_rolex, N, 'rolex', 'Llama')

In [ ]:
model_response(formatted_spaceship, N, 'spaceship', 'Llama')